In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Preprocessing
!ls

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1vvRy60J06fZq-dj8EPVTkgHU4gq8OCqU/Capstone-KPMG/Preprocessing
Attractions.ipynb		 I90.ipynb
combine_data.ipynb		 map_request_api.py
Combine.ipynb			 num_EV_chargers_around_gas.ipynb
Data				 OBSOLETE_Geo.ipynb
EV_station_count_distance.ipynb  OBSOLETE_remoteDataList
gas_station_w_traffic.csv	 __pycache__
HighwayExitDistance.ipynb	 traffic_population_crime_combination.ipynb


In [2]:
import pandas as pd
from map_request_api import *

# Generate gas station to exit data for stations next to I5 Exit 1A to Exit 108

In [67]:
# Load exit data
exits = pd.read_csv('Data/I5Exits.csv')


# I90 part 2
# start_lat = 46.940413018818525
# start_long = -119.99523354565559
# end_lat = 47.696911869042218
# end_long = -117.04170811912506

# I90 part 1
# start_lat = 47.592719119437461
# start_long = -122.31213053243302
# end_lat = 46.940413018818525
# end_long = -119.99523354565559


# I82
# start_lat = 46.93112
# start_long = -120.50267
# end_lat = 45.94989
# end_long = -119.33755

# # I5 part 6
# end_lat = exits[exits['attributes.ExitNumber']== '274'].iloc[0]['geometry.y']
# end_long = exits[exits['attributes.ExitNumber']== '274'].iloc[0]['geometry.x']
# start_lat = exits[exits['attributes.ExitNumber']== '221'].iloc[0]['geometry.y']
# start_long = exits[exits['attributes.ExitNumber']== '221'].iloc[0]['geometry.x']


# I5 part 5
# end_lat = exits[exits['attributes.ExitNumber']== '221'].iloc[0]['geometry.y']
# end_long = exits[exits['attributes.ExitNumber']== '221'].iloc[0]['geometry.x']
# start_lat = exits[exits['attributes.ExitNumber']== '183'].iloc[0]['geometry.y']
# start_long = exits[exits['attributes.ExitNumber']== '183'].iloc[0]['geometry.x']


# # I5 part 4
# end_lat = exits[exits['attributes.ExitNumber']== '183'].iloc[0]['geometry.y']
# end_long = exits[exits['attributes.ExitNumber']== '183'].iloc[0]['geometry.x']
# start_lat = exits[exits['attributes.ExitNumber']== '163'].iloc[0]['geometry.y']
# start_long = exits[exits['attributes.ExitNumber']== '163'].iloc[0]['geometry.x']


# I5 part 3
# end_lat = exits[exits['attributes.ExitNumber']== '163'].iloc[0]['geometry.y']
# end_long = exits[exits['attributes.ExitNumber']== '163'].iloc[0]['geometry.x']
# start_lat = exits[exits['attributes.ExitNumber']== '143'].iloc[0]['geometry.y']
# start_long = exits[exits['attributes.ExitNumber']== '143'].iloc[0]['geometry.x']


# I5 part 2
# end_lat = exits[exits['attributes.ExitNumber']== '143'].iloc[0]['geometry.y']
# end_long = exits[exits['attributes.ExitNumber']== '143'].iloc[0]['geometry.x']
# start_lat = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.y']
# start_long = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.x']

# I5 part1 
end_lat = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.y']
end_long = exits[exits['attributes.ExitNumber']== '108'].iloc[0]['geometry.x']
start_lat = exits[exits['attributes.ExitNumber']== '1 A'].iloc[0]['geometry.y']
start_long = exits[exits['attributes.ExitNumber']== '1 A'].iloc[0]['geometry.x']

In [68]:
end_lat, end_long

(47.04458239749142, -122.84329811931372)

In [69]:
# get route/corridor
url = build_search_url_latlong(str(start_lat), str(start_long), str(end_lat), str(end_long))
result = get_result(url)
sessionId = result['route']['sessionId']
get_route_distance(result)

107.6892

In [70]:
# Filter gas stations within 10 mi range of  Corridor
WIDTH = 10
gas_station = pd.read_csv('Data/wa_gas_stations.csv')
remoteDataList = getRemoteDataList(gas_station,'Latitude','Longitude','name')

result = post_corridor_search_request(sessionId, remoteDataList, WIDTH)
print(result['resultsCount'])
filtered_gas_stations = pd.DataFrame(result['searchResults'])

323


In [71]:
# covert exits to remoteData Format
exitRomoteDataList = getRemoteDataList(exits, 'geometry.y', 'geometry.x', 'attributes.ExitNumber')

In [72]:
# for each gas station near I-90, compute its distance to closest exit
dist_to_exit = []
for gas_r in range(filtered_gas_stations.shape[0]):
  gas_data = filtered_gas_stations.iloc[gas_r]
  dist, exit_n, exit_p = get_closest_exit(gas_data['shapePoints'], exitRomoteDataList)
  # print(gas_data['resultNumber'], dist)
  dist_to_exit.append([gas_data['key'],gas_data['shapePoints'][0],gas_data['shapePoints'][1],exit_n,exit_p[0],exit_p[1],dist])
dist_to_exit = pd.DataFrame(dist_to_exit)
dist_to_exit.columns = ['gas_key','gas_lat','gas_long','exit_name','exit_lat','exit_long','distance_to_nearest_exit']

In [73]:
dist_to_exit = dist_to_exit.rename(columns={"distance": "distance_to_nearest_exit"})
dist_to_exit['interstate'] = 'I5'
dist_to_exit.to_csv('Data/gas_station_w_10mi_to_exit_I5_1.csv')
dist_to_exit

,gas_key,gas_lat,gas_long,exit_name,exit_lat,exit_long,distance_to_nearest_exit,interstate
0,1101,46.009816,-122.845820,30,46.003649,-122.843998,0.4355,I5
1,832,47.044527,-122.845099,108,47.044582,-122.843298,0.0850,I5
2,1783,45.911880,-122.745401,21,45.917607,-122.749961,0.4528,I5
3,1033,45.908302,-122.744860,21,45.902501,-122.740547,0.4518,I5
4,488,46.710777,-122.974268,82,46.717613,-122.976120,0.4810,I5
...,...,...,...,...,...,...,...,...
318,1145,45.605016,-122.502823,27,45.601265,-122.550991,2.3454,I5
319,1654,45.602043,-122.503394,27,45.601265,-122.550991,2.3041,I5
320,1260,45.596987,-122.504483,27,45.601265,-122.550991,2.2702,I5
321,2001,46.324594,-122.732775,52,46.326987,-122.910691,8.5000,I5


In [74]:
dist_to_exit.distance_to_nearest_exit.max()

8.5